# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

In [3]:
!pip install --upgrade pip

# Libraries

In [4]:
!pip install torchsummaryX==1.1.0 wandb --quiet

In [5]:
!pip install --upgrade pip

In [6]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [7]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

In [8]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"jiayihuang2001","key":"d27e2930f4739944e1c7f6f2e539e1bc"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory '/root/.kaggle': File exists


In [9]:
!pip install slugify

In [10]:
!kaggle competitions download -c 11785-hw1p2-f24

11785-hw1p2-f24.zip: Skipping, found more recently modified local copy (use --force to force download)


In [11]:
!unzip -qo ./11785-hw1p2-f24.zip 

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [30]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100",fraction = 0.1): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        content_dir =os.path.join(root,"11785-f24-hw1p2")
        partition_dir = os.path.join(content_dir,partition)
        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir = os.path.join(partition_dir,"mfcc")
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = os.path.join(partition_dir,"transcript")

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_list = sorted(os.listdir(self.mfcc_dir))
        mfcc_names = mfcc_list
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcipt_list = sorted(os.listdir(self.transcript_dir))
        transcript_names = transcipt_list

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []
        idx = 0
        file_idx_dict ={}
        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,mfcc_names[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc_n = mfcc - np.mean(mfcc,axis=0,keepdims=True)
            idx_last = idx
            idx = len(mfcc_n)+idx
            for inx in range(idx_last,idx):
                file_idx_dict[inx] = i 
    
        #   Load the corresponding transcript
            mfccs_p = np.pad(mfcc_n,((context,context),(0,0)),mode ="constant")
            self.mfccs.append(mfccs_p)
        #   Load the corresponding transcript
            transcript  = np.load(os.path.join(self.transcript_dir,transcript_names[i]))[1:-1] # Remove [SOS] and [EOS] from the transcript
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.transcripts.append(transcript)
        self.file_idx_dict =file_idx_dict
        # NOTE:
        # Each mfcc is of shape T1 x 28, T2 x 28, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 28 (Where T = T1 + T2 + ...)
        self.mfccs = np.concatenate(self.mfccs)

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.transcripts)  # Ensure length is based on transcripts
        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers
        phoneme_dict ={}
        for index,ph in enumerate(self.phonemes):
            phoneme_dict[ph]=index
        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        self.transcripts = np.array([phoneme_dict[ph] for ph in self.transcripts])
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        file = self.file_idx_dict[ind]
        start_ind = ind + 2*self.context * file
        end_ind = ind + 2*self.context +1 + 2*self.context * file
        # Retrieve the frames with context, handling potential edge cases
        frames = self.mfccs[start_ind:end_ind]
        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data
        frames     = torch.FloatTensor(frames) # Convert to tensors
        phonemes   = torch.tensor(self.transcripts[ind])

        return frames, phonemes
    


class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, context=0, partition= "test-clean"): # Feel free to add more arguments

        self.context = context
        content_dir =os.path.join(root,"11785-f24-hw1p2")
        partition_dir = os.path.join(content_dir,partition)
        self.mfcc_dir = os.path.join(partition_dir,"mfcc")
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        self.mfccs = []
        idx = 0
        file_idx_dict = {}
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,mfcc_names[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc_n = mfcc - np.mean(mfcc,axis=0,keepdims=True)
            idx_last = idx
            idx = len(mfcc_n)+idx
            for inx in range(idx_last,idx):
                file_idx_dict[inx] = i
        #   Load the corresponding transcript
            mfccs_p = np.pad(mfcc_n,((context,context),(0,0)),mode ="constant")
            self.mfccs.append(mfccs_p)
        self.mfccs = np.concatenate(self.mfccs)
        self.length = len(self.mfccs)
        self.file_idx_dict = file_idx_dict

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        file= self.file_idx_dict[ind]
        start_ind = ind + 2*self.context * file
        end_ind = ind + 2*self.context +1 + 2*self.context * file
        # Retrieve the frames with context, handling potential edge cases
        frames = self.mfccs[start_ind:end_ind]
        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data
        frames     = torch.FloatTensor(frames) # Convert to tensors
        return frames
        

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

# Create Datasets

In [31]:
config = {
    "model"         : "6-MLP",
    'epochs'        : 10,
    'batch_size'    : 1024,
    'context'       : 50,
    'init_lr'       : 1e-3,
    'architecture'  : '6_layer'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

In [32]:

root =os.getcwd()
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset(root,context=config["context"])
# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data =  AudioDataset(root,context=config["context"],partition="dev-clean")
# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(root,context=config["context"])

In [33]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  1024
Context        :  50
Input size     :  2828
Output symbols :  42
Train dataset samples = 36091157, batches = 35246
Validation dataset samples = 1928204, batches = 1884
Test dataset samples = 2196138, batches = 2145


In [34]:
# Testing code to check if your data loaders are working

# so only one batch here
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([1024, 2828]) torch.Size([1024])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [35]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 3072),
            torch.nn.BatchNorm1d(3072),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(3072, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 1024), 
            torch.nn.BatchNorm1d(1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,128),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, output_size),
            torch.nn.Softmax(dim=1)
        )

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [36]:
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                [2828, 3072]         [1024, 3072]             8,690.69                 8.69
1_BatchNorm1d                 [3072]         [1024, 3072]                 6.14                 0.00
2_ReLU                             -         [1024, 3072]                    -                    -
3_Dropout                          -         [1024, 3072]                    -                    -
4_Linear                [3072, 2048]         [1024, 2048]             6,293.50                 6.29
5_BatchNorm1d                 [2048]         [1024, 2048]                 4.10                 0.00
6_ReLU                             -         [1024, 2048]                    -                    -
7_Linear                [2048, 1024]         [1024, 1024]             2,098.18                 2.10

In [37]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate,
scheduler1 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max= config['epochs'])
# including but not limited to StepLR, MultiStep, CosineAnnealing, CosineAnnealingWithWarmRestarts, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [38]:
torch.cuda.empty_cache()
gc.collect()

1589

In [39]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [40]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [41]:
wandb.login(key="bc022b99e5a39b97fc6ae8c641ab328e9f52d2e6") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [42]:
# Create your wandb run
run = wandb.init(
    name    = "first-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

In [43]:
# run = wandb.init(
#     project="hw1p2",
#     job_type="model-training",
#     name=config['model'],
#     config=config
#     )
# ## Retreiving the model

# # Getting the latest version of the artifact
# artifact = run.use_artifact('{}:latest'.format(config['model']))
# # Downloading the artifact
# artifact_dir = artifact.download()
# # Loading the model
# model_dict = torch.load(os.path.join(artifact_dir, 'Model'))



# # Loading weights
# model.load_state_dict(model_dict['model_state_dict'])
# # Loading optimizer state
# optimizer.load_state_dict(model_dict['optimizer_state_dict'])

In [44]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/kaggle/working/wandb/run-20240909_213458-187qcer5/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")

best_acc =0.60
for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)
    scheduler1.step()
    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    # Updating the model version
    if train_acc > best_acc:
        best_acc = train_acc
        # Saving the model and optimizer states

        torch.save({
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict()
              }, "Model")

        # Creating Artifact

        model_artifact = wandb.Artifact(config['model'], type='model')

        # Adding model file to Artifact

        model_artifact.add_file("Model")

        # Saving Artifact to WandB

        run.log_artifact(model_artifact)

    
    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    


Epoch 1/10


Train:   0%|          | 0/35246 [00:00<?, ?it/s]

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)
            
            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits,dim=1)
            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            test_predictions.append(predicted_phonemes)
            # TODO
        print(len(torch.concatenate(test_predictions)))
    return torch.concatenate(test_predictions)

In [ ]:
predictions = test(model, test_loader)

In [ ]:
print(predictions.shape)

In [ ]:
### Create CSV file with predictions

with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, PHONEMES[predictions[i]]))

In [ ]:
### Finish your wandb run
run.finish()

In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11785-hw1p2-f24 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle